In [24]:
from get_mnist_features import get_gp_mnist_data

import numpy as np

import gpflow
import tensorflow as tf

In [25]:
mnist_train = np.load('data/mnist_train.npy')
X_train = np.load('data/X_train.npy')
y_train = np.load('data/y_train.npy')
mnist_test = np.load('data/mnist_test.npy')
X_test = np.load('data/X_test.npy')
y_test = np.load('data/y_test.npy')

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

num_classes = 10
num_features = 128

X_train = X_train.astype('float64').reshape(-1,num_features)
y_train = y_train.astype('float64').reshape(-1,1)

(60000, 128)
(60000,)
(10000, 128)
(10000,)


In [26]:
trained_vars = np.load("optimised_vars.npy").item()
values = np.load("values.npy").item()
print(values.keys())
print(trained_vars.keys())
for key in trained_vars.keys():
    print(trained_vars[key].shape)
    print(np.array_equal(values[key], trained_vars[key]))

FileNotFoundError: [Errno 2] No such file or directory: 'optimised_vars.npy'

In [29]:
X_train = X_train[:59000]
y_train = y_train[:59000]
kernel = gpflow.kernels.Matern32(num_features) + gpflow.kernels.White(num_features, variance=0.01)
likelihood = gpflow.likelihoods.MultiClass(num_classes)
Z=X_train[::1000].copy()
num_latent=num_classes
whiten=True
q_diag=True
m = gpflow.models.SVGP(X_train, y_train,
                       kern=kernel,
                       likelihood=likelihood,
                       Z=Z,
                       num_latent=num_latent,
                       whiten=whiten,
                       q_diag=q_diag)

m.kern.white.variance.trainable = False
m.feature.trainable = False
opt = gpflow.train.ScipyOptimizer()

m.compile()
# opt.minimize(m)

In [30]:
saver = tf.train.Saver()
sess = gpflow.session_manager.get_default_session()
saver.restore(sess, "models/gp.ckpt")
print("Model restored.")

INFO:tensorflow:Restoring parameters from models/gp.ckpt


NotFoundError: Key 137/SVGP/q_mu/unconstrained not found in checkpoint
	 [[Node: save_11/RestoreV2_21 = RestoreV2[dtypes=[DT_DOUBLE], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_11/Const_0_0, save_11/RestoreV2_21/tensor_names, save_11/RestoreV2_21/shape_and_slices)]]

Caused by op 'save_11/RestoreV2_21', defined at:
  File "/Users/sebastian/anaconda/envs/python3/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/sebastian/anaconda/envs/python3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/sebastian/anaconda/envs/python3/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/sebastian/anaconda/envs/python3/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/sebastian/anaconda/envs/python3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line

In [ ]:
a = list(zip(m.trainable_parameters, m.read_trainables()))
for x,y in a:
    print("param:", x)
    print("train:", y)
    print()
# param_dict = {p[0].full_name.replace('SGPR', 'SGPU'): p[1] for p in zip(vfe.trainable_parameters, vfe.read_trainables())}